In [1]:

import plot_functions
import compare_clusterings
import process_data
import visualizations

from compare_clusterings import *
from process_data import *
from plot_functions import *
from visualizations import *

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3
import matplotlib.animation as animation

In [3]:
import traj_generator

In [4]:
import numpy as np
import plotly.graph_objs as go

In [5]:
import numpy as np

# Parameters
num_points = 100
num_groups = 4  # Allow for all strategies to be represented
space_size = 10
timesteps = 200
noise_level = 0.1
group_change_interval = 50

# Initialize points
positions = np.random.rand(num_points, 3) * space_size  # Random 3D positions
velocities = np.random.rand(num_points, 3) - 0.5        # Random initial velocities
group_assignments = np.random.randint(0, num_groups, num_points)  # Random group assignments

# Shared parameters for group motions
group_motion_types = np.random.choice([1, 2, 3, 4], size=num_groups)  # Assign strategies (1, 2, 3, or 4)
group_base_velocities = np.random.rand(num_groups, 3) - 0.5  # Shared velocities for Strategy 1
group_directions = np.random.rand(num_groups, 3) - 0.5       # Shared directions for Strategy 3
group_directions = group_directions / np.linalg.norm(group_directions, axis=1)[:, np.newaxis]  # Normalize directions

# Circular motion parameters
circular_directions = [np.random.rand(3) for _ in range(num_groups)]  # Rotation axes for Strategy 4
circular_directions = [d / np.linalg.norm(d) for d in circular_directions]  # Normalize rotation axes

# Initialize storage for the simulation
trajectory_data = np.zeros((num_points, timesteps, 4))  # (x, y, z, cluster)

def rotation_matrix_around_axis(axis, angle):
    """Create a 3D rotation matrix around a given axis."""
    axis = axis / np.linalg.norm(axis)
    cos_a = np.cos(angle)
    sin_a = np.sin(angle)
    u = axis
    return np.array([
        [cos_a + u[0]**2 * (1 - cos_a), u[0] * u[1] * (1 - cos_a) - u[2] * sin_a, u[0] * u[2] * (1 - cos_a) + u[1] * sin_a],
        [u[1] * u[0] * (1 - cos_a) + u[2] * sin_a, cos_a + u[1]**2 * (1 - cos_a), u[1] * u[2] * (1 - cos_a) - u[0] * sin_a],
        [u[2] * u[0] * (1 - cos_a) - u[1] * sin_a, u[2] * u[1] * (1 - cos_a) + u[0] * sin_a, cos_a + u[2]**2 * (1 - cos_a)],
    ])

def update_positions(positions, velocities, group_assignments, noise_level):
    """Update the positions of the points."""
    group_centers = np.zeros((num_groups, 3))
    group_counts = np.zeros(num_groups)
    
    # Compute group centers (for Strategy 2: Group Center Attraction)
    for i, pos in enumerate(positions):
        group = group_assignments[i]
        group_centers[group] += pos
        group_counts[group] += 1
    
    for g in range(num_groups):
        if group_counts[g] > 0:
            group_centers[g] /= group_counts[g]  # Average position for each group

    # Update positions based on group strategy
    for i, pos in enumerate(positions):
        group = group_assignments[i]
        strategy = group_motion_types[group]
        
        if strategy == 1:  # Shared Group Velocity Vector
            velocities[i] = group_base_velocities[group] + noise_level * (np.random.rand(3) - 0.5)
        
        elif strategy == 2:  # Group Center Attraction
            direction_to_center = group_centers[group] - pos
            if np.linalg.norm(direction_to_center) > 1e-6:  # Avoid division by zero
                velocities[i] = direction_to_center / np.linalg.norm(direction_to_center) + noise_level * (np.random.rand(3) - 0.5)
        
        elif strategy == 3:  # Shared Direction with Noise
            velocities[i] = group_directions[group] + noise_level * (np.random.rand(3) - 0.5)
            velocities[i] /= np.linalg.norm(velocities[i])  # Normalize to maintain direction
        
        elif strategy == 4:  # Circular Motion
            axis = circular_directions[group]
            angle = 0.1  # Constant angular step size
            rotation_matrix = rotation_matrix_around_axis(axis, angle)
            velocities[i] = np.dot(rotation_matrix, velocities[i])
        
        # Update position
        positions[i] += velocities[i]

    return positions

# Simulation
for t in range(timesteps):
    positions = update_positions(positions, velocities, group_assignments, noise_level)
    
    # Store positions and cluster assignments for each point at this timestep
    for i, pos in enumerate(positions):
        trajectory_data[i, t, :3] = pos  # Store x, y, z
        trajectory_data[i, t, 3] = group_assignments[i]  # Store cluster ID
    
    # Change group memberships periodically
    if t % group_change_interval == 0:
        group_assignments = np.random.randint(0, num_groups, num_points)

# The trajectory_data array contains the required format
# [[[x1_t1, y1_t1, z1_t1, cluster1_t1], [x1_t2, y1_t2, z1_t2, cluster1_t2], ...],
#  [[x2_t1, y2_t1, z2_t1, cluster2_t1], [x2_t2, y2_t2, z2_t2, cluster2_t2], ...], ...]

trajectories = trajectory_data


In [6]:
num_timesteps = len(trajectories[0])
num_particles = len(trajectories)
groups_list = np.array(trajectories)[:,:,-1]
num_groups = len(np.unique(np.array(groups_list).flatten()))

In [8]:
%matplotlib notebook

#a = plot_functions.plot_traj_labels_plt(trajectory_data, save_video=True, interval=100, filename="new_example.mp4")

In [9]:
#restructure for redpandda

In [10]:
flattened_data = []
trajectories = trajectories.tolist()
for obj_id, particle_data in enumerate(trajectories):
    for t, record in enumerate(particle_data):
        
        flattened_data.append([obj_id, t] + record)

df = pd.DataFrame(flattened_data, columns=['obj_id', 't', 'x', 'y', 'z', 'label'])

print(df)

       obj_id    t         x          y          z  label
0           0    0  7.812605   5.979638   3.608453    3.0
1           0    1  7.856689   6.827952   4.136109    2.0
2           0    2  7.834504   7.645067   4.712156    2.0
3           0    3  7.749048   8.422952   5.334725    2.0
4           0    4  7.603951   9.153965   6.001484    2.0
...       ...  ...       ...        ...        ...    ...
19995      99  195 -0.952493  70.021699  59.774227    3.0
19996      99  196 -0.767269  70.896814  60.221292    3.0
19997      99  197 -0.642426  71.780488  60.672438    3.0
19998      99  198 -0.461547  72.644177  61.142889    3.0
19999      99  199 -0.357396  73.507200  61.637201    3.0

[20000 rows x 6 columns]


# pair plot 1

In [12]:
import redpandda_general
traj_array, point_array, frames_count, n_objects = redpandda_general.prepare_data_from_df(df)

dist_matrices = redpandda_general.get_distance_matrices(traj_array)
delta_matrices = redpandda_general.get_delta_matrices(dist_matrices)
average_delta_matrix = redpandda_general.calculate_average_delta_matrix(delta_matrices)

std_delta_matrix = redpandda_general.get_std_matrices(dist_matrices)

stddv_matrices = redpandda_general.get_stddv(dist_matrices)

In [13]:
from timestep_clustering import compute_timstep_clustering, apply_rpt_change_detection

In [14]:
from sklearn.preprocessing import StandardScaler

In [15]:
def full_workflow(delta_matrices, scale_embedding = False, change_point_detection=True, clustering_method="hdbscan"):
    embedding = compute_timstep_clustering(delta_matrices, title_type="Delta-matrices", dim_red="umap", metric="wasserstein", plot_dim_red=True, kde_plot=True, plot_heatmap=True)
    if isinstance(embedding, tuple):
        embedding = embedding[0]


    if scale_embedding:
        from sklearn.preprocessing import StandardScaler
        embedding = StandardScaler().fit_transform(embedding)

    if change_point_detection:
        change_points = apply_rpt_change_detection(embedding[0], pen=5)

    else:
        if clustering_method == "hdbscan":

            pass



In [27]:
compute_timstep_clustering(delta_matrices, title_type="Delta-matrices", dim_red="umap", metric="wasserstein", plot_dim_red=True, kde_plot=True, plot_heatmap=True)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

(array([[ 20.626616 ,  -5.4545846],
        [ 20.64463  ,  -5.4670324],
        [ 20.612959 ,  -5.4316683],
        [ 20.747812 ,  -5.5563884],
        [ 20.852257 ,  -5.6424036],
        [ 20.912973 ,  -5.693009 ],
        [ 21.069777 ,  -5.812481 ],
        [ 21.083223 ,  -5.817598 ],
        [ 21.21614  ,  -5.9092693],
        [ 21.37204  ,  -6.018781 ],
        [ 21.441437 ,  -6.0654545],
        [ 21.60705  ,  -6.1800227],
        [ 21.803616 ,  -6.320275 ],
        [ 21.90379  ,  -6.3844595],
        [ 22.180582 ,  -6.5586715],
        [ 22.354815 ,  -6.6746645],
        [ 22.632189 ,  -6.841807 ],
        [ 22.770739 ,  -6.9334083],
        [ 22.915249 ,  -7.017734 ],
        [ 23.113247 ,  -7.1264977],
        [ 23.162374 ,  -7.154314 ],
        [ 23.498123 ,  -7.3527346],
        [ 23.682476 ,  -7.470942 ],
        [ 23.903006 ,  -7.5650883],
        [ 24.043736 ,  -7.6207814],
        [ 24.257812 ,  -7.763212 ],
        [ 24.39679  ,  -7.861913 ],
        [ 24.607252 ,  -7.98

In [18]:
embedding = compute_timstep_clustering(delta_matrices, title_type="Delta-matrices", dim_red="umap", metric="wasserstein", plot_dim_red=True, kde_plot=True, plot_heatmap=True)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [19]:
import hdbscan
clusterer = hdbscan.HDBSCAN(min_cluster_size=5)  # Adjusting min_cluster_size is usually all you need
labels = clusterer.fit_predict(embedding[0])

In [20]:
labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3])

In [21]:
change_points = apply_rpt_change_detection(embedding[0], pen=5)

In [22]:
import clustering_functions

In [23]:
timestep_clusters = clustering_functions.cluster_timesteps_change_points(delta_matrices, change_points, "affinity")

0.0452
0.0136
0.0090
0.0136


In [24]:
timestep_clusters

[{'start': 0,
  'end': 50,
  'clustering': array([4, 2, 5, 5, 5, 1, 5, 0, 3, 4, 0, 0, 0, 5, 3, 0, 3, 4, 1, 0, 0, 5,
         2, 3, 3, 2, 5, 2, 0, 3, 5, 5, 3, 1, 5, 0, 3, 2, 3, 3, 0, 5, 1, 5,
         0, 0, 5, 5, 4, 0, 3, 2, 0, 3, 4, 4, 0, 3, 1, 4, 3, 5, 3, 0, 5, 5,
         3, 0, 1, 2, 4, 4, 0, 2, 1, 0, 5, 0, 3, 3, 5, 0, 4, 0, 0, 3, 3, 0,
         3, 0, 4, 5, 0, 1, 0, 5, 3, 5, 0, 1])},
 {'start': 51,
  'end': 100,
  'clustering': array([3, 4, 4, 3, 2, 2, 0, 1, 3, 4, 2, 4, 4, 3, 6, 3, 4, 3, 3, 1, 2, 3,
         4, 3, 3, 3, 2, 2, 4, 4, 2, 2, 6, 6, 3, 2, 6, 4, 4, 4, 1, 3, 6, 2,
         3, 2, 0, 0, 4, 2, 4, 2, 3, 2, 5, 3, 2, 3, 6, 3, 2, 0, 4, 1, 4, 3,
         4, 3, 4, 2, 2, 3, 4, 4, 4, 1, 2, 3, 3, 3, 0, 2, 3, 1, 1, 3, 3, 3,
         2, 3, 5, 0, 4, 4, 2, 4, 4, 2, 4, 4])},
 {'start': 101,
  'end': 150,
  'clustering': array([5, 0, 0, 4, 4, 4, 0, 5, 0, 4, 0, 5, 0, 1, 4, 4, 0, 5, 5, 0, 2, 0,
         3, 1, 4, 4, 0, 2, 5, 4, 4, 2, 4, 0, 5, 4, 0, 5, 0, 4, 0, 1, 0, 0,
         1, 0, 5, 0, 3, 0,

In [25]:
timestep_clusters_from_clustering = clustering_functions.cluster_timesteps_from_timestep_clustering(delta_matrices, labels, "affinity")

0.0400
0.0132
0.0446
0.0213


In [26]:
timestep_clusters_from_clustering

[{'indices': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
         17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
         34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]),
  'clustering': array([18,  7,  6,  6,  6, 10,  6,  0,  1, 18, 11,  2, 32,  6,  3, 19, 17,
         18, 10,  4,  5,  6,  7, 27,  8,  7,  6,  7,  9,  1,  7,  6, 17, 10,
          6, 11, 17,  7,  3, 17, 12,  6, 10,  6, 13, 32,  6,  6, 18, 14,  3,
          7, 15, 31, 18, 18, 16, 17, 10, 18, 17,  6,  3, 19,  6,  6, 31, 20,
         10,  7, 18, 18, 32,  7, 10, 21,  6, 22,  3, 17,  6, 23, 18, 24, 25,
          8,  3, 26, 27, 28, 18,  6, 29, 10, 30,  6, 31,  6, 32, 10])},
 {'indices': array([50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66,
         67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83,
         84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]),
  'clustering': array([5, 2, 2, 5, 1, 1, 0, 6, 5,